# Speculative Decoding

SGLang now provides an EAGLE-based (EAGLE-2/EAGLE-3) speculative decoding option. Our implementation aims to maximize speed and efficiency and is considered to be among the fastest in open-source LLM engines.

### Performance Highlights

Please see below for the huge improvements on throughput for LLaMA-Instruct 3.1 8B tested on MT bench that can be achieved via EAGLE3 decoding.
For further details please see the [EAGLE3 paper](https://arxiv.org/pdf/2503.01840).

| Method | Throughput (tokens/s) |
|--------|----------------|
| SGLang (w/o speculative, 1x H100) | 158.34 tokens/s |
| SGLang + EAGLE-2 (1x H100) | 244.10 tokens/s |
| SGLang + EAGLE-3 (1x H100) | 373.25 tokens/s |

## EAGLE Decoding

To enable EAGLE speculative decoding the following parameters are relevant:
* `speculative_draft_model_path`: Specifies draft model. This parameter is required.
* `speculative_num_steps`: Depth of autoregressive drafting. Increases speculation range but risks rejection cascades. Default is 5.
* `speculative_eagle_topk`: Branching factor per step. Improves candidate diversity, will lead to higher acceptance rate, but more lead to higher memory/compute consumption. Default is 4.
* `speculative_num_draft_tokens`: Maximum parallel verification capacity. Allows deeper tree evaluation but will lead to higher GPU memory usage. Default is 8.

These parameters are the same for EAGLE-2 and EAGLE-3.

You can find the best combinations of these parameters with [bench_speculative.py](https://github.com/sgl-project/sglang/blob/main/scripts/playground/bench_speculative.py).

In the documentation below, we set `--cuda-graph-max-bs` to be a small value for faster engine startup. For your own workloads, please tune the above parameters together with `--cuda-graph-max-bs`, `--max-running-requests`, `--mem-fraction-static` for the best performance. 

### EAGLE-2 decoding

You can enable EAGLE-2 decoding by setting `--speculative-algorithm EAGLE` and choosing an appropriate model.

In [1]:
from sglang.test.doc_patch import launch_server_cmd
from sglang.utils import wait_for_server, print_highlight, terminate_process

import openai

[2025-11-05 01:23:13] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.


[2025-11-05 01:23:13] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.


[2025-11-05 01:23:13] INFO utils.py:164: NumExpr defaulting to 16 threads.


In [2]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 3 \
    --speculative-eagle-topk 4 --speculative-num-draft-tokens 16 --cuda-graph-max-bs 8 --log-level warning
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-11-05 01:23:21] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
[2025-11-05 01:23:21] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
[2025-11-05 01:23:21] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2025-11-05 01:23:24] WARNING server_args.py:1165: Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-11-05 01:23:24] WARNING server_args.py:1521: Overlap scheduler is disabled because of using eagle3 or standalone speculative decoding.
[2025-11-05 01:23:24] INFO trace.py:52: opentelemetry package is not installed, tracing disabled


[2025-11-05 01:23:34] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
[2025-11-05 01:23:34] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
[2025-11-05 01:23:34] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2025-11-05 01:23:36] INFO trace.py:52: opentelemetry package is not installed, tracing disabled


[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0


...

Capturing batches (bs=2 avail_mem=53.70 GB):  50%|█████     | 2/4 [00:20<00:20, 10.30s/it]

Capturing batches (bs=1 avail_mem=53.68 GB):  75%|███████▌  | 3/4 [00:21<00:05,  5.77s/it]

Capturing batches (bs=1 avail_mem=53.68 GB): 100%|██████████| 4/4 [00:25<00:00,  6.25s/it]


Capturing batches (bs=1 avail_mem=53.56 GB):   0%|          | 0/4 [00:00<?, ?it/s]

Capturing batches (bs=1 avail_mem=53.56 GB): 100%|██████████| 4/4 [00:00<00:00, 13.30it/s]


In [3]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

In [4]:
terminate_process(server_process)

### EAGLE-2 Decoding with `torch.compile`

You can also enable `torch.compile` for further optimizations and optionally set `--torch-compile-max-bs`:


In [5]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --mem-fraction 0.6 \
            --enable-torch-compile --torch-compile-max-bs 2 --log-level warning
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-11-05 01:25:41] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
[2025-11-05 01:25:41] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
[2025-11-05 01:25:41] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2025-11-05 01:25:44] WARNING server_args.py:1165: Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-11-05 01:25:44] WARNING server_args.py:1521: Overlap scheduler is disabled because of using eagle3 or standalone speculative decoding.
[2025-11-05 01:25:44] INFO trace.py:52: opentelemetry package is not installed, tracing disabled


[2025-11-05 01:25:50] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
[2025-11-05 01:25:50] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
[2025-11-05 01:25:50] INFO utils.py:164: NumExpr defaulting to 16 threads.
[2025-11-05 01:25:50] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
[2025-11-05 01:25:50] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
[2025-11-05 01:25:50] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2025-11-05 01:25:52] INFO trace.py:52: opentelemetry package is not installed, tracing disabled
[2025-11-05 01:25:52] INFO trace.py:52: opentelemetry package is not installed, tracing disabled


[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0


...

Capturing batches (bs=3 avail_mem=53.06 GB):  25%|██▌       | 1/4 [00:07<00:22,  7.64s/it]

Capturing batches (bs=2 avail_mem=37.93 GB):  50%|█████     | 2/4 [00:08<00:07,  3.53s/it]

Capturing batches (bs=1 avail_mem=37.89 GB):  75%|███████▌  | 3/4 [00:08<00:02,  2.11s/it]

Capturing batches (bs=1 avail_mem=37.89 GB): 100%|██████████| 4/4 [00:14<00:00,  3.68s/it]


Capturing batches (bs=1 avail_mem=30.96 GB): 100%|██████████| 4/4 [00:00<00:00, 74.06it/s]


In [6]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

In [7]:
terminate_process(server_process)

### EAGLE-2 Decoding via Frequency-Ranked Speculative Sampling

By employing a truncated high-frequency token vocabulary in the draft model, Eagle speculative decoding reduces `lm_head` computational overhead while accelerating the pipeline without quality degradation. For more details, checkout [the paper](https://arxiv.org/pdf/arXiv:2502.14856).

In our implementation, set `--speculative-token-map` to enable the optimization. You can get the high-frequency token in FR-Spec from [this model](https://huggingface.co/thunlp/LLaMA3-Instruct-8B-FR-Spec). Or you can obtain high-frequency token by directly downloading these token from [this repo](https://github.com/thunlp/FR-Spec/tree/main?tab=readme-ov-file#prepare-fr-spec-vocabulary-subset).

Thanks for the contribution from [Weilin Zhao](https://github.com/Achazwl) and [Zhousx](https://github.com/Zhou-sx). 

In [8]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Meta-Llama-3-8B-Instruct --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-LLaMA3-Instruct-8B --speculative-num-steps 5 \
    --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --speculative-token-map thunlp/LLaMA3-Instruct-8B-FR-Spec/freq_32768.pt \
    --mem-fraction 0.7 --cuda-graph-max-bs 2 --dtype float16  --log-level warning
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-11-05 01:27:35] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
[2025-11-05 01:27:35] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
[2025-11-05 01:27:35] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2025-11-05 01:27:37] WARNING model_config.py:880: Casting torch.bfloat16 to torch.float16.
[2025-11-05 01:27:37] WARNING server_args.py:1165: Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-11-05 01:27:37] WARNING server_args.py:1521: Overlap scheduler is disabled because of using eagle3 or standalone speculative decoding.
[2025-11-05 01:27:37] INFO trace.py:52: opentelemetry package is not installed, tracing disabled


[2025-11-05 01:27:37] Casting torch.bfloat16 to torch.float16.


[2025-11-05 01:27:42] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
[2025-11-05 01:27:42] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
[2025-11-05 01:27:42] INFO utils.py:164: NumExpr defaulting to 16 threads.
[2025-11-05 01:27:43] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
[2025-11-05 01:27:43] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
[2025-11-05 01:27:43] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2025-11-05 01:27:44] INFO trace.py:52: opentelemetry package is not installed, tracing disabled
[2025-11-05 01:27:45] INFO trace.py:52: opentelemetry package is not installed, tracing disabled


...

Capturing batches (bs=3 avail_mem=42.02 GB):  25%|██▌       | 1/4 [00:04<00:12,  4.19s/it]

Capturing batches (bs=2 avail_mem=58.45 GB):  50%|█████     | 2/4 [00:04<00:04,  2.07s/it]

Capturing batches (bs=1 avail_mem=58.38 GB):  75%|███████▌  | 3/4 [00:05<00:01,  1.25s/it]

Capturing batches (bs=1 avail_mem=58.38 GB): 100%|██████████| 4/4 [00:07<00:00,  1.79s/it]


Capturing batches (bs=1 avail_mem=57.75 GB): 100%|██████████| 4/4 [00:00<00:00, 108.94it/s]


In [9]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

In [10]:
terminate_process(server_process)

### EAGLE-3 Decoding

You can enable EAGLE-3 decoding by setting `--speculative-algorithm EAGLE3` and choosing an appropriate model.

In [11]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-3.1-8B-Instruct  --speculative-algorithm EAGLE3 \
    --speculative-draft-model-path jamesliu1/sglang-EAGLE3-Llama-3.1-Instruct-8B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 32 --mem-fraction 0.6 \
        --cuda-graph-max-bs 2 --dtype float16 --log-level warning
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-11-05 01:29:04] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
[2025-11-05 01:29:04] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
[2025-11-05 01:29:04] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2025-11-05 01:29:06] WARNING model_config.py:880: Casting torch.bfloat16 to torch.float16.
[2025-11-05 01:29:06] WARNING server_args.py:1165: Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-11-05 01:29:06] WARNING server_args.py:1521: Overlap scheduler is disabled because of using eagle3 or standalone speculative decoding.
[2025-11-05 01:29:06] INFO trace.py:52: opentelemetry package is not installed, tracing disabled


[2025-11-05 01:29:12] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
[2025-11-05 01:29:12] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
[2025-11-05 01:29:12] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2025-11-05 01:29:13] INFO trace.py:52: opentelemetry package is not installed, tracing disabled


[2025-11-05 01:29:14] Casting torch.bfloat16 to torch.float16.


...

Capturing batches (bs=3 avail_mem=57.26 GB):  25%|██▌       | 1/4 [00:02<00:07,  2.58s/it]

Capturing batches (bs=2 avail_mem=57.22 GB):  50%|█████     | 2/4 [00:02<00:02,  1.26s/it]

Capturing batches (bs=1 avail_mem=57.17 GB):  75%|███████▌  | 3/4 [00:03<00:00,  1.27it/s]

Capturing batches (bs=1 avail_mem=57.17 GB): 100%|██████████| 4/4 [00:05<00:00,  1.28s/it]


Capturing batches (bs=1 avail_mem=57.02 GB): 100%|██████████| 4/4 [00:00<00:00, 47.71it/s]


In [12]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

In [13]:
terminate_process(server_process)

## Multi Token Prediction

We support [MTP(Multi-Token Prediction)](https://arxiv.org/pdf/2404.19737) in SGLang by using speculative decoding. We use Xiaomi/MiMo-7B-RL model as example here (deepseek mtp usage refer to [deepseek doc](../basic_usage/deepseek.md#multi-token-prediction))

In [14]:
server_process, port = launch_server_cmd(
    """
    python3 -m sglang.launch_server --model-path XiaomiMiMo/MiMo-7B-RL --host 0.0.0.0 --trust-remote-code \
    --speculative-algorithm EAGLE --speculative-num-steps 1 --speculative-eagle-topk 1 --speculative-num-draft-tokens 2 \
    --mem-fraction 0.5 --log-level warning
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-11-05 01:30:12] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
[2025-11-05 01:30:12] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
[2025-11-05 01:30:12] INFO utils.py:164: NumExpr defaulting to 16 threads.


A new version of the following files was downloaded from https://huggingface.co/XiaomiMiMo/MiMo-7B-RL:
- configuration_mimo.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
[2025-11-05 01:30:15] WARNING dynamic_module_utils.py:485: A new version of the following files was downloaded from https://huggingface.co/XiaomiMiMo/MiMo-7B-RL:
- configuration_mimo.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


[2025-11-05 01:30:15] WARNING server_args.py:1165: Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-11-05 01:30:15] WARNING server_args.py:1521: Overlap scheduler is disabled because of using eagle3 or standalone speculative decoding.
[2025-11-05 01:30:15] INFO trace.py:52: opentelemetry package is not installed, tracing disabled


[2025-11-05 01:30:23] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
[2025-11-05 01:30:23] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
[2025-11-05 01:30:23] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2025-11-05 01:30:25] INFO trace.py:52: opentelemetry package is not installed, tracing disabled


...

Capturing batches (bs=1 avail_mem=59.71 GB): 100%|██████████| 4/4 [00:00<00:00,  7.70it/s]


[2025-11-05 01:31:19] SPECULATIVE_MOE_RUNNER_BACKEND is not initialized, using auto backend
Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:00,  4.84it/s]


Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:00<00:00,  8.03it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:00<00:00,  7.65it/s]



Capturing batches (bs=1 avail_mem=58.86 GB): 100%|██████████| 4/4 [00:00<00:00, 64.59it/s]


In [15]:
import requests

url = f"http://localhost:{port}/v1/chat/completions"

data = {
    "model": "XiaomiMiMo/MiMo-7B-RL",
    "messages": [{"role": "user", "content": "What is the capital of France?"}],
}

response = requests.post(url, json=data)
print_highlight(response.json())

In [16]:
terminate_process(server_process)

## References

EAGLE process is as follows:

- Within EAGLE the draft model predicts the next feature vector, i.e. the last hidden state of the original LLM, using the feature sequence $(f_1, ..., f_k)$ and the token sequence $(t_2, ..., t_{k+1})$. 
- The next token is then sampled from $p_{k+2}=\text{LMHead}(f_{k+1})$. Afterwards, the two sequences are extended in a tree style—branching out multiple potential continuations, with the branching factor per step controlled by the `speculative_eagle_topk` parameter—to ensure a more coherent connection of context, and are given as input again.
- EAGLE-2 additionally uses the draft model to evaluate how probable certain branches in the draft tree are, dynamically stopping the expansion of unlikely branches. After the expansion phase, reranking is employed to select only the top `speculative_num_draft_tokens` final nodes as draft tokens.
- EAGLE-3 removes the feature prediction objective, incorporates low and mid-layer features, and is trained in an on-policy manner.

This enhances drafting accuracy by operating on the features instead of tokens for more regular inputs and passing the tokens from the next timestep additionally to minimize randomness effects from sampling. Furthermore the dynamic adjustment of the draft tree and selection of reranked final nodes increases acceptance rate of draft tokens further. For more details see [EAGLE-2](https://arxiv.org/abs/2406.16858) and [EAGLE-3](https://arxiv.org/abs/2503.01840) paper.


For guidance how to train your own EAGLE model please see the [EAGLE repo](https://github.com/SafeAILab/EAGLE/tree/main?tab=readme-ov-file#train).